In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from ib_async import IB, util

In [ ]:
util.startLoop()

host = "127.0.0.1"
port = 7496
clientId = 6

TWS_CONNECTION = IB().connect(host, port, clientId, timeout=30)

In [ ]:
from ib_async import IB, Contract, Option
from ib_async.ticker import Ticker
from typing import List
from enum import Enum
from abc import ABC, abstractmethod


class MarketDataType(Enum):
    """
    Market Data Type enum for IBKR TWS
    https://interactivebrokers.github.io/tws-api/market_data_type.html
    """

    LIVE = 1
    FROZEN = 2
    DELAYED = 3
    DELAYED_FROZEN = 4


class RealTimeSource(ABC):
    pass


class IbkrTws(RealTimeSource):
    def __init__(self, ib=None):
        self.ib = ib
        self.ib.reqMarketDataType(MarketDataType.LIVE.value)

    def get_accounts(self):
        return self.ib.managedAccounts()

    def get_positions_for_account(self, account_id: str):
        return self.ib.positions(account=account_id)

    def get_quotes(self, con_ids: List[str]):
        pass

    def get_trades(self):
        return self.ib.trades()

    def get_executions(self):
        return self.ib.executions()


ibkr_realtime = IbkrTws(ib=TWS_CONNECTION)
trades_ibasync_objects = ibkr_realtime.get_trades()
# ibkr_realtime.get_executions()
# accounts = ibkr_realtime.get_accounts()
# positions_ibasync_objects = ibkr_realtime.get_positions_for_account(accounts[0])

In [ ]:
from ib_async import util

real_time_trades_df = util.df(trades_ibasync_objects)

In [ ]:
# real_time_trades_df

In [ ]:
from ngv_reports_ibkr.dtype_exporter import export_dtypes_all

# export_dtypes_all(real_time_trades_df, '../data', prefix='ibkr_tws_realtime_trades', only_export=['md'])

In [ ]:
from ngv_reports_ibkr.expand_contract_columns import expand_all_trade_columns, expand_fills_and_logs

expanded_df = expand_all_trade_columns(real_time_trades_df)
expanded_df = expand_fills_and_logs(expanded_df, fills_col="fills", log_col="log")

export_dtypes_all(expanded_df, "../data", prefix="ibkr_tws_realtime_trades_interpreted", only_export=["md"])

mask_traded_mcl = expanded_df.symbol == "MCL"

account_ids = expanded_df.account_id.unique()
# account_ids

pd.set_option("display.max_rows", None)

mask_account_id = expanded_df.account_id.str.contains("16")
examaple_df = expanded_df[mask_account_id & mask_traded_mcl]

# examaple_df.to_parquet('../data/ibkr_tws_realtime_trades_interpreted_example.parquet')
examaple_df.to_csv("../data/ibkr_tws_realtime_trades_interpreted_example.csv")